In [14]:
#all imports
import carla
import math


# Build connection to the CARLA server
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)


# world = client.get_world()
# spawn_points = world.get_map().get_spawn_points()

# vehicle_bp = world.get_blueprint_library().filter('*model3*')
# start_point = spawn_points[0]
# vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [37]:

import random
import pygame # We will be using this for manual control

pygame 2.6.1 (SDL 2.28.4, Python 3.9.22)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [19]:
# Load layered map for Town 01 with minimum layout plus buildings and parked vehicles
world = client.load_world('Town01_Opt', carla.MapLayer.Buildings | carla.MapLayer.ParkedVehicles)

# Toggle all buildings off
world.unload_map_layer(carla.MapLayer.Buildings)

# Toggle all buildings on   
world.load_map_layer(carla.MapLayer.Buildings)

In [20]:
# changed the weather to a clear noon.

weather = carla.WeatherParameters(
    cloudiness=0.0,
    precipitation=0.0,
    sun_altitude_angle=10.0,
    sun_azimuth_angle = 90.0,
    precipitation_deposits = 0.0,
    wind_intensity = 0.0,
    fog_density = 0.0,
    wetness = 0.0, 
)
world.set_weather(weather)

In [21]:
# First let's get the blueprint library and the spawn points for our world.
# Depending on your Carla version and the map chosen, you get different actors
# and spawn points respectively

bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points()

In [22]:
# I am spawning an Audi etron here. You can check out the blueprint library
# to spawn your vehicle of choice. Also we spawn in a random safe point 79

vehicle_bp = bp_lib.find('vehicle.audi.etron')
ego_vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[79]) 

In [23]:
# Let's position the spectator just behind the vehicle
# Carla.Transform has two parameters - Location and Rotation. We use this to
# to position the spectator by going 4 metres behind and 2.5 metres above the 
# ego_vehicle

spectator = world.get_spectator()
transform = carla.Transform(ego_vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),ego_vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [25]:
# If you want to position the your_actor with just the coordinates, 
# you can use the below codes.
# location = carla.Location(x=0, y=0, z=30)
# rotation = carla.Rotation(roll=0, pitch=-30, yaw=180)
# transform = carla.Transform(location, rotation)
# your_actor.set_transform(transform)

In [26]:
## NPCs Spawning

In [28]:
import random 
# Let's add some traffic. I am spawning 200 vehicle actors at random spawn_points
# (includes all kinds of automobiles). The method try_spawn_actor makes sure
# you can spawn it without messing with any existing actors.

for i in range(200):  
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))


#Set traffic in motion
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True) # This makes all the vehicles function in autopilot
ego_vehicle.set_autopilot(False) # Allows you to drive the ego_vehicle manually

In [ ]:
## Vehicle Properties

In [30]:
## Vehicle PHYSICS property

# Create Wheels Physics Control
front_left_wheel  = carla.WheelPhysicsControl(tire_friction=2.0, damping_rate=1.5, max_steer_angle=70.0, long_stiff_value=1000)
front_right_wheel = carla.WheelPhysicsControl(tire_friction=2.0, damping_rate=1.5, max_steer_angle=70.0, long_stiff_value=1000)
rear_left_wheel   = carla.WheelPhysicsControl(tire_friction=3.0, damping_rate=1.5, max_steer_angle=0.0,  long_stiff_value=1000)
rear_right_wheel  = carla.WheelPhysicsControl(tire_friction=3.0, damping_rate=1.5, max_steer_angle=0.0,  long_stiff_value=1000) # Reducing friction increases idle throttle 

wheels = [front_left_wheel, front_right_wheel, rear_left_wheel, rear_right_wheel]

# Change Vehicle Physics Control parameters of the vehicle
physics_control = ego_vehicle.get_physics_control()
physics_control.torque_curve = [carla.Vector2D(x=0, y=400), carla.Vector2D(x=1300, y=600)]
physics_control.max_rpm = 10000
physics_control.moi = 1.0
physics_control.damping_rate_full_throttle = 0.0
physics_control.use_gear_autobox = True
physics_control.gear_switch_time = 0.5
physics_control.clutch_strength = 10
physics_control.mass = 10000
physics_control.drag_coefficient = 0.25
physics_control.steering_curve = [carla.Vector2D(x=0, y=1), carla.Vector2D(x=100, y=1), carla.Vector2D(x=300, y=1)]
physics_control.use_sweep_wheel_collision = True
physics_control.wheels = wheels

# Apply Vehicle Physics Control for the vehicle
vehicle.apply_physics_control(physics_control)

In [31]:
## Manual Control

In [33]:
! pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 67.0 MB/s eta 0:00:001m60.8 MB/s eta 0:00:01


In [38]:
pygame.init() #initialising

# Set up the Pygame display
size = (640, 480)
pygame.display.set_caption("CARLA Manual Control")
screen = pygame.display.set_mode(size)

# Set up the control object and loop until the user exits the script
control = carla.VehicleControl()
clock = pygame.time.Clock()
done = False

while not done:
    # Get keyboard input and handle it
    keys = pygame.key.get_pressed() 
    
    # The values are directly from the manual control code which we ran in the
    # last post. Just implemented just the throttle, brake and steer
    # You can add reverse and gear shifting features directly from that code

    if keys[pygame.K_UP] or keys[pygame.K_w]:
        control.throttle = min(control.throttle + 0.05, 1.0)
    else:
        control.throttle = 0.0

    if keys[pygame.K_DOWN] or keys[pygame.K_s]:
        control.brake = min(control.brake + 0.2, 1.0)
    else:
        control.brake = 0.0

    if keys[pygame.K_LEFT] or keys[pygame.K_a]:
        control.steer = max(control.steer - 0.05, -1.0)
    elif keys[pygame.K_RIGHT] or keys[pygame.K_d]:
        control.steer = min(control.steer + 0.05, 1.0)
    else:
        control.steer = 0.0

    control.hand_brake = keys[pygame.K_SPACE]

    # Apply the control to the ego vehicle and tick the simulation
    ego_vehicle.apply_control(control)
    world.tick()

    # Update the display and check for the quit event
    pygame.display.flip()
    pygame.display.update()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True

    # Sleep to ensure consistent loop timing
    clock.tick(60)

In [2]:
print(vehicle_bp[0].id)

vehicle.tesla.model3


In [3]:
# get spectator
spectator = world.get_spectator() #Retrieves the spectator camera actor from the world

spectator_pos = carla.Transform(start_point.location + carla.Location(x=20,y=10,z=4),
                                carla.Rotation(yaw = start_point.rotation.yaw -155))
spectator.set_transform(spectator_pos)

In [4]:
# this gets the car's position on the map 
vehicle_transform = vehicle.get_transform()

In [5]:
#send vehicle off
vehicle.set_autopilot(True)

In [7]:
# utility script of destruction just in case

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [8]:
#function to claculate sides (x,y) of a right-angle triangle

def calculate_sides(hypotenuse, angle):
  """
  Calculates the two sides of a right triangle given the hypotenuse and an angle.

  Args:
    hypotenuse: The length of the hypotenuse of the triangle.
    represents the distance we need to be from the car
    angle: The angle of the triangle in degrees.
    represents the yaw angle of the car we need to be aligned with 

  Returns:
    A tuple containing the lengths of the two sides of the triangle.
    which are delta x and y
  """

  # Convert the angle to radians
  angle_radians = math.radians(angle)

  # Calculate the opposite side using the sine function
  opposite_side = hypotenuse * math.sin(angle_radians)

  # Calculate the adjacent side using the cosine function
  adjacent_side = hypotenuse * math.cos(angle_radians)

  return opposite_side, adjacent_side

In [9]:
# follow the car
# here we subtract the delta x and y to be behind 
metres_distance = 5
vehicle_transform = vehicle.get_transform()
y,x = calculate_sides(metres_distance, vehicle_transform.rotation.yaw )

spectator_pos = carla.Transform(vehicle_transform.location + carla.Location(x=-x,y=-y,z=5 ),
                                        carla.Rotation( yaw = vehicle_transform.rotation.yaw,pitch = -25))
spectator.set_transform(spectator_pos)            

In [10]:
# view head on
# here we add delta x and y to be in front of the car
# but also add 180 degree yaw to "turn around" towards the car 
metres_distance = 15
vehicle_transform = vehicle.get_transform()
y,x = calculate_sides(metres_distance, vehicle_transform.rotation.yaw )

spectator_pos = carla.Transform(vehicle_transform.location + carla.Location(x=x,y=y,z=5 ),
                                        carla.Rotation( yaw = vehicle_transform.rotation.yaw - 180,pitch = -25))
spectator.set_transform(spectator_pos)      

In [11]:
vehicle_bp = world.get_blueprint_library().filter('*police*')
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [12]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.set_autopilot(False)

In [13]:
# get spectator
spectator = world.get_spectator()
spectator_pos = carla.Transform(start_point.location + carla.Location(x=20,y=10,z=4),
                                carla.Rotation(yaw = 0, pitch = -90))
spectator.set_transform(spectator_pos)